### 설정 + 필요한 모델 다운로드

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 13.9 MB/s 
     |████████████████████████████████| 448 kB 66.2 MB/s 


In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 14.9 MB/s 
     |████████████████████████████████| 6.5 MB 52.6 MB/s 
     |████████████████████████████████| 895 kB 99.0 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 81.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install git+https://github.com/SKT-AI/KoBART #egg=kobart

  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-req-build-pl23v3rp
  Running command git clone -q https://github.com/SKT-AI/KoBART /tmp/pip-req-build-pl23v3rp
     |████████████████████████████████| 132 kB 13.5 MB/s 
     |████████████████████████████████| 814 kB 57.0 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 1.9 MB 79.1 MB/s 
     |████████████████████████████████| 134 kB 80.1 MB/s 
     |████████████████████████████████| 829 kB 71.6 MB/s 
     |████████████████████████████████| 3.3 MB 74.8 MB/s 
     |████████████████████████████████| 1.1 MB 81.2 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 144 kB 64.3 MB/s 
     |████████████████████████████████| 271 kB 96.6 MB/s 
     |████████████████████████████████| 79 kB 10.6 MB/s 
     |████████████████████████████████| 8.6 MB 37.3 MB/s 
     |████████████████████████████████| 127 kB 63.8 MB/s 
  Created wheel for kobar

In [69]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 13.3 MB/s 
     |████████████████████████████████| 144 kB 70.1 MB/s 
     |████████████████████████████████| 181 kB 75.2 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=20198f6fec9461fe1be4c438eac42c82f1f986b996647570ed74a5d6f1c1ed7a
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [70]:
import torch
import os 
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb

from google.colab import drive
import re
from konlpy.tag import Okt
import requests
from bs4 import BeautifulSoup

from itertools import combinations
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.data import Dataset, DataLoader,random_split

from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
from transformers import AutoModel, BartModel

In [5]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


### 원본 데이터 불러오기

In [6]:
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
csv_test = pd.read_csv("/content/drive/MyDrive/NLP/sports_news_data - sports_news_data.csv")

In [8]:
csv_test.head(20)

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15
5,"긴급 수혈된 바르사 NO.9, 1년 반 만에 떠난다… ‘EPL행 유력’",[스포탈코리아] FC 바르셀로나는 새 시즌을 앞두고 선수단 정리가 한창이다. 잉여 ...,2021-07-15
6,"[김남구의 유럽통신] 황의조, 손흥민 소속사와 손잡다… CAA Base와 계약",[스포탈코리아=파리(프랑스)] 황의조(지롱댕 드 보르도)가 한국 선수로는 3번째로 ...,2021-07-15
7,"""메시 종신은 축복!""…스폰서 5년 더 보장, 바르셀로나 함박웃음",[스포탈코리아] 리오넬 메시(34)가 FC바르셀로나에 남는다. 연봉을 절반 삭감하지...,2021-07-15
8,"[오피셜] 눈물 흘렸던 '37세 전설' 로번, 두 번째 현역 은퇴 발표",[스포탈코리아] 네덜란드 축구스타 아르연 로번(37)이 현역 은퇴를 밝혔다. \n\...,2021-07-15
9,"100세' 메시팬 할아버지, 748골 수기 작성…메시도 감사 인사",[스포탈코리아] 리오넬 메시(34)는 프로 데뷔하고 748골을 터뜨렸다. 전산화 하...,2021-07-15


### 전처리
- 중복 및 결측치 제거
- 크롤링 상에서 생긴 쓸모 없는 문구 처리


In [9]:
def preprocess(string):
    str_list = string.split('\n\n') # \n\n 기준으로 분리
    str_list[0] = str_list[0].replace('[스포탈코리아] ', '') # [스포탈코리아] 삭제
    
    result = []
    for strg in str_list:
        strg = re.sub(r'\([^)]*\)|\[[^)]*\]|\<[^)]*\>', ' ', strg) # 괄호와 그 안에 문자 제거

        if '기자' in strg: # 기자 이름이 들어간 경우 기자이름 삭제
            strg = strg.split('기자')[-1]
        if '다.' in strg: # 문장 단위로 분리
            strg = strg.split('. ')
            for s in strg:
                if s != '':
                    result.append(s)
            
    return result

In [10]:
def special(str_list):
    regex = r"[^가-힣a-zA-Z0-9 ]" # 특수 문자 제거
    for i in range(len(str_list)):
        str_list[i] = re.sub(regex, '', str_list[i])

    return str_list

In [11]:
def special_title(string):
    regex = r"[^가-힣a-zA-Z0-9 ]" # 특수 문자 제거
    string = re.sub(regex, '', string)

    return string

In [42]:
trimmed_data=csv_test.dropna().copy()
  
idx = trimmed_data['CONTENT'].drop_duplicates().index
trimmed_data = trimmed_data.loc[idx]

idx = trimmed_data['TITLE'].drop_duplicates().index
trimmed_data = trimmed_data.loc[idx]

trimmed_data['CONTENT'] = trimmed_data['CONTENT'].apply(preprocess)
trimmed_data['CONTENT'] = trimmed_data['CONTENT'].apply(special)
trimmed_data['TITLE'] = trimmed_data['TITLE'].apply(special_title)


In [47]:
idx_list=[]
for i in range(len(trimmed_data)):
  if len(trimmed_data['CONTENT'].iloc[i]) < 4:
    idx_list.append(trimmed_data.iloc[i].name)

abandoned_data = trimmed_data.loc[idx_list]
trimmed_data = trimmed_data.drop(idx_list)

8995


In [45]:
print(trimmed_data['CONTENT'].loc[3189])

['  맨체스터 유나이티드  수비수 라파엘 바란이 사타구니 부상으로 당분간 결장할 예정이다', ' 맨유는 13일 오전  구단 공식 채널을 통해 라파엘 바란이 UEFA 네이션스리그 결승전에서 당한 사타구니 부상으로 재활에 들어갔다', '바란은 몇 주 동안 아웃될 것이다라고 전했다']


In [48]:
len(trimmed_data)

8358

In [14]:
trimmed_data['CONTENT'].iloc[8991]

['성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다',
 '김지수는 192cm84kg의 체격에 제공권이 좋고 시야가 넓어 대인 방어와 패스에 능하다',
 ' 김지수는 성남FC U15 U18에서 활약하며 성남의 유망주로 이름을 알렸으며 꾸준히 연령별 대표팀에도 이름을 올리고 있다',
 'U15 크로아티아 국제 축구대회와 AFC U16 챔피언십 예선 참여 2021년 경기도 꿈나무 축구대회 우승 2021년 제29회 백록기 전국 고교 축구대회에서 우승 주역으로 베스트 영플레이어상을 수상한 김지수는 성남FC U18 풍생고에서 성장 가능성이 큰 선수로 꼽혀왔다 구단은 유소년 육성에 집중 투자하면서 구단 유스 프로 계약의 증가와 구단 최초 준프로 계약 등 점차 가시적인 성과를 내고 있으며 꾸준한 유소년 시스템 투자와 유망주 발굴을 통해 구단의 지속적 성장의 기틀을 다질 예정이다 김남일 감독은 풍생고 경기를 꾸준히 보면서 김지수를 눈여겨봤었다',
 '책임감 있고 안정적인 경기 운영을 하는 것이 눈에 띄었다',
 '김지수가 성남에서 첫 프로 도전을 하는 만큼 열심히 준비해서 경기장에서 본인의 잠재력을 마음껏 펼치길 바란다라고 밝혔다 김지수는 이번 1차 전지훈련에서 프로팀 형들과 훈련을 한 것도 영광이었는데 바로 계약까지 하게 된 게 아직 꿈만 같다',
 '성남 유스 출신 선수라는 책임감이 생겼다',
 '2차 전지훈련 동안 선수 형들에게 많이 배우고 열심히 준비해서 팀에 도움이 되고 싶다라고 밝혔다 그는 성남FC 협력병원인 분당베스트병원에서 메디컬테스트를 마쳤고 2차 전지훈련지인 부산 기장으로 출발하여 본격적인 2022시즌 준비에 나선다']

- 띄어쓰기 및 불용어 처리

In [15]:
# 한국어 불용어 리스트 크롤링


url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ranks.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


# Korean stop words: 677


In [16]:

okt = Okt()
for i in tqdm(range(len(trimmed_data))):
  temp_data = okt.morphs(trimmed_data["TITLE"].iloc[i])
  temp_list = []

  for word in temp_data:
    if word in stop_words: continue
    temp_list.append(word)
  
  trimmed_data["TITLE"].iloc[i] = " ".join(temp_list)

  temp_list = []
  for sentence in trimmed_data["CONTENT"].iloc[i]:
    temp_data = okt.morphs(sentence)
    temp_sentecne_list = []
    
    for word in temp_data:
      if word in stop_words: continue
      temp_sentecne_list.append(word)
    
    temp_sentence = " ".join(temp_sentecne_list)
    temp_list.append(temp_sentence)
  
  trimmed_data["CONTENT"].iloc[i] = temp_list
  


100%|██████████| 8995/8995 [05:27<00:00, 27.48it/s]


In [17]:
trimmed_data.head()

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,[유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나...,2021-07-15
1,디 마리아 없다 유로 X 코파 베스트 11 이탈리아 만 7 명,"[지난달 시작 된 유로 코파 아메리카 11일 끝 막 내렸다, 이탈리아 는 결승전 잉...",2021-07-15
2,슈퍼컴퓨터 예측 맨시티 우승 맨유 4 위 토트넘 은 6 위,"[새 시즌 시작 하기도 전 슈퍼컴퓨터 예상 한 순위 나왔다, 영국 매체 스포츠 바이...",2021-07-15
3,이재성 완벽한 프로 마인츠 서 성공할 킬 디렉터 애정 듬뿍 응원,"[홀슈타인 킬 우베 스토 버 디렉터 이재성 향 해 응원 메시지 띄웠다, 이재성 은 ...",2021-07-15
4,홈킷 딴판 바르사 팬 NEW 웨이 셔츠 호평 가장 좋아하는 색,"[FC 바르셀로나 새 시즌 원정 유니폼 공개 했다, 팬 은 만족스럽다는 반응 이다,...",2021-07-15


In [18]:
data_row = trimmed_data.iloc[0]
print(data_row)

text = data_row['CONTENT']
print(text)

for i, sentence in enumerate(text):
  print(i)
  print(sentence)

TITLE                              스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아
CONTENT       [유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나...
PUBLISH_DT                                           2021-07-15
Name: 0, dtype: object
['유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나온 판정 논란 정심 이라고 공언 했다', '지난 8일 잉글랜드 덴마크 는 유로 2020 4 강 결승 티켓 두고 격돌 했다', '연장 접전 끝 잉글랜드 21 이겼다', '경기 후 논란 불거졌다', '11 팽팽 하던 연장 전반 12분 라 힘 스털링 드리블 돌파 하던 중 요아킴 멜레 마티아스 옌센 사이 넘어졌다', '심판 은 곧장 페널티 스팟 찍었다', '비디오 판독 실과 의견 나눈 뒤 에도 원심 유지 했다', '페널티킥 얻은 잉글랜드 는 해리 케인 실축 했지만 흐른 볼 밀어 넣어 결승 티켓 따냈다', '장면 두고 갑론 박 펼쳐졌다', '스털링 은 경기 후 인터뷰 명백한 페널티킥 이라고 주장 했지만 전문가 의견 은 달랐다', '조제 모리뉴 AS 로마 감독 아르 센 벵거 전 아스널 감독 은 페널티킥 아니다고 입 모았다', '많은 이야기 흘러나오는 가운데 유로 2020 심판 위원장 로세티 오심 아니라는 입장 내놨다', '로세티 위원장 은 14일 영국 매체 가디언 인터뷰 주심 은 5 번 수비수 주목 했다', '수비수 볼 터치 하지 않았다고 봤다', '멜레 오른쪽 다리 스털링 오른쪽 다리 접촉 한 확인 했다', '접촉 강도 논 할 수 있지만 는 항상 심판 의사결정 과정 중심 되길 바란다고 밝혔다', '로세티 위원장 심판 대표 해 의견 냈지만 오심 이라고 생각 하는 받아들일지는 미지수 다', '스털링 은 평소 에도 다이빙 논란 시 달려왔고 많은 머릿속 다이버 라는 인식 가득하기 때문 이다']
0
유럽 축구 연맹 유

In [19]:
max_sentence_num = 0
for i in range(len(trimmed_data)):
  sentence_num = len(trimmed_data["CONTENT"].iloc[i])
  max_sentence_num = max(sentence_num,max_sentence_num)

print(max_sentence_num)

115


#Extractive summarization - Matchsum

### Dataset & Dataloader 생성

In [55]:
def control_input_ids(input_ids_tensor,length,cls_token_num,sep_token_num,pad_token_num):
  cur_length = len(input_ids_tensor)
  cls_token = torch.tensor([cls_token_num])
  sep_token = torch.tensor([sep_token_num])

  if cur_length+2 > length:
    input_ids_tensor = input_ids_tensor[:length-2]  # 길이가 넘치면 자른다
    return torch.cat([cls_token,input_ids_tensor,sep_token])
  else:
    input_ids_tensor = torch.cat([cls_token,input_ids_tensor,sep_token])
    padding_list = torch.tensor([pad_token_num]*(length - cur_length -2)) # 길이가 모자라면 padding token 을 채운다
    return torch.cat([input_ids_tensor,padding_list])

In [56]:
def custom_collate_fn(samples):
  
  text_ids = torch.empty(0,512)
  labels_ids = torch.empty(0,32)
  for sample in samples:
    text_ids = torch.cat([text_ids,sample['text_input_ids'].unsqueeze(0)],dim=0) 
    labels_ids = torch.cat([labels_ids,sample['labels_input_ids'].unsqueeze(0)],dim=0)

  sentence_input_ids = [sample['sentence_input_ids'] for sample in samples]
  nn.utils.rnn.pad_sequence(sentence_input_ids,batch_first=True,padding_value = 1)

  return dict(text_input_ids = text_ids.to(torch.int64), labels_input_ids = labels_ids.to(torch.int64), sentence_input_ids = sentence_input_ids)

In [57]:
class CustomDataset(Dataset):
  def __init__(
      self, data, tokenizer,
      text_max_token_len = 512,
      summary_max_token_len = 32
        ):
    self.tokenizer = tokenizer
    self.data = data
    self.text_max_token_len = text_max_token_len
    self.summary_max_token_len = summary_max_token_len
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index):
    cls_token_num = 0
    sep_token_num = 2
    pad_token_num = 1
    
    data_row = self.data.iloc[index]
    text = data_row['CONTENT']
    
    total_text_ids = torch.tensor([])
    sentence_input_ids = torch.empty(0,32)

    for sentence in text:
      text_encoding_sentence = self.tokenizer(
          sentence,return_tensors = "pt",add_special_tokens=False)
      sentence_indiv_input_ids = text_encoding_sentence['input_ids'].flatten()
      total_text_ids = torch.cat([total_text_ids,sentence_indiv_input_ids])

      sentence_indiv_input_ids = control_input_ids(sentence_indiv_input_ids,self.summary_max_token_len,cls_token_num,sep_token_num,pad_token_num)
      sentence_indiv_input_ids = sentence_indiv_input_ids.unsqueeze(0)
      sentence_input_ids = torch.cat([sentence_input_ids,sentence_indiv_input_ids],dim=0)
    
    sentence_input_ids = sentence_input_ids.to(torch.int64)
    total_text_ids = control_input_ids(total_text_ids,self.text_max_token_len,cls_token_num,sep_token_num,pad_token_num)    
    total_text_ids = total_text_ids

    labels = data_row['TITLE']
    summary_encoding = self.tokenizer(
        labels,
        add_special_tokens = False,
        return_tensors = "pt"
    )

    labels_ids = summary_encoding['input_ids'].flatten()
    labels_ids = control_input_ids(labels_ids,self.summary_max_token_len,cls_token_num,sep_token_num,pad_token_num)

    return dict(text_input_ids = total_text_ids, labels_input_ids = labels_ids, sentence_input_ids = sentence_input_ids)


In [58]:
tokenizer = get_kobart_tokenizer()

using cached model. /content/.cache/kobart_base_tokenizer_cased_cf74400bce.zip


In [59]:
whole_dataset = CustomDataset(trimmed_data,tokenizer)

train_set_num = 7000
train_dataset , valid_dataset = random_split(whole_dataset, [train_set_num,len(trimmed_data)-train_set_num])
train_dataloader = DataLoader(train_dataset, batch_size = 2, shuffle=True,collate_fn = custom_collate_fn)
valid_dataloader =  DataLoader(valid_dataset, batch_size = 2, shuffle=False,collate_fn = custom_collate_fn)

### Matchsum

- 평가 metric -> rdass
- 기본적으로 모델에 스코어가 높은 5개의 단일 문장을 뽑고 뽑인 문장으로 만들어진 조합 가운데서 스코어가 높은 조합을 golden summary로 선정
- loss 는 margin ranking loss 사용


In [25]:
def get_score(doc,label,answer):
  score_1 = torch.cosine_similarity(doc,answer,dim=0)
  score_2 = torch.cosine_similarity(label,answer,dim=0)
  return score_1+score_2

In [63]:
def get_candidate_id(doc_emb,summary_emb,batch_sentence_id, candidate_num, extract_model,device):
    cls_token = torch.tensor([0]).to(device)
    sep_token = torch.tensor([2]).to(device)
    candidate_ids = torch.empty([0,candidate_num,128]).to(device)
    
    for batch_idx, sentence_id_tensor in enumerate(batch_sentence_id):
      sentence_id_tensor = sentence_id_tensor.to(device)
      out = extract_model.forward(sentence_id_tensor)  #sentence_id_tensor = [문장 갯수,32개의 토큰]
      hidden_states = out['last_hidden_state'][:,0,:] # [문장 갯수,token 갯수 ,768 dim_vec]
      score_list= []
      
      for i in range(hidden_states.shape[0]):
        score = get_score(doc = doc_emb[batch_idx,:], label = summary_emb[batch_idx,:], answer = hidden_states[i,:])
        score_list.append((score,i))
      
      score_list.sort(key = lambda x: x[0],reverse=True)
      idx_list = [idx for _,idx in score_list][:5]
    
      # get candidate summaries
      # here is for CNN/DM: truncate each document into the 5 most important sentences (using BertExt), 
      # then select any 2 or 3 sentences to form a candidate summary, so there are C(5,2)+C(5,3)=20 candidate summaries.
      # if you want to process other datasets, you may need to adjust these numbers according to specific situation.
      indices = list(combinations(idx_list, 2))
      indices += list(combinations(idx_list, 3))
      if len(idx_list) < 2:
          indices = [idx_list]
    
      # get score for each candidate summary and sort them in descending order
      score = []
      for i in indices:
          i = list(i)
          i.sort()
          # write dec
          dec = torch.tensor([]).to(device)
          for j in i:
              sent = sentence_id_tensor[j]
              sent = sent[1:]
              sep_token_idx = 0
              for token_idx in range(len(sent)):
                if sent[token_idx] == 2: break
                else:sep_token_idx += 1
              sent = sent[:sep_token_idx]
              dec = torch.cat([dec,sent],dim=0)
          
          dec = torch.cat([cls_token,dec,sep_token],dim=0)
          dec = dec.to(torch.int64)
          dec_out = extract_model.forward(input_ids = dec.unsqueeze(0))
          score.append((dec, get_score(doc_emb[batch_idx,:],summary_emb[batch_idx,:], dec_out['last_hidden_state'][0,0,:])))
      
      score.sort(key=lambda x : x[1], reverse=True)
      score = score[:candidate_num]
      
      candidate_ids_ind= torch.empty(0,128).to(device)
      for k,_ in score:
        dec = k
        if len(dec) < 128:
          padding_list = torch.tensor([1]*(128-len(dec))).to(device)
          dec = torch.cat([k,padding_list],dim=0)
        else:
          dec = dec[:128]

        candidate_ids_ind = torch.cat([candidate_ids_ind,dec.unsqueeze(0)],dim = 0)

      candidate_ids = torch.cat([candidate_ids,candidate_ids_ind.unsqueeze(0)],dim = 0)

    return candidate_ids.to(torch.int64)

In [66]:
class MatchSum(nn.Module):  
    def __init__ (self, encoder, candidate_num, device,hidden_size=768):
        super(MatchSum, self).__init__()
        
        self.hidden_size = hidden_size
        self.candidate_num  = candidate_num
        self.encoder = encoder
        self.device = device

    def forward(self, text_id, summary_id,list_of_sentence_id):
        
        batch_size = text_id.size(0)
        pad_id = 1 

        # get document embedding
        input_mask = ~(text_id == pad_id)
        out = self.encoder(text_id, attention_mask=input_mask)['last_hidden_state'] # last layer
        doc_emb = out[:, 0, :]
        assert doc_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size]
        
        # get summary embedding
        input_mask = ~(summary_id == pad_id)
        out = self.encoder(summary_id, attention_mask=input_mask)['last_hidden_state'] # last layer
        summary_emb = out[:, 0, :]
        assert summary_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size]

        # get summary score
        summary_score = torch.cosine_similarity(summary_emb, doc_emb, dim=-1)

        # get candidate embedding
        candidate_id = get_candidate_id(doc_emb,summary_emb,list_of_sentence_id, self.candidate_num, self.encoder,self.device) #[batch_size , candidate_num, token_num]
        candidate_id = candidate_id.view(-1, candidate_id.size(-1)) 
        input_mask = ~(candidate_id == pad_id)
        out = self.encoder(candidate_id, attention_mask=input_mask)['last_hidden_state'] 
        candidate_emb = out[:, 0, :].view(batch_size, self.candidate_num, self.hidden_size)  # [batch_size, candidate_num, hidden_size]
        assert candidate_emb.size() == (batch_size, self.candidate_num, self.hidden_size)
        
        # get candidate score
        doc_emb = doc_emb.unsqueeze(1).expand_as(candidate_emb)
        score = torch.cosine_similarity(candidate_emb, doc_emb, dim=-1) # [batch_size, candidate_num]
        golden_list = torch.argmax(score,dim=1)
        assert score.size() == (batch_size, self.candidate_num)

        return {'score': score, 'summary_score': summary_score, 
                'golden_summary':torch.cat([candidate_id[0,:].unsqueeze(0),candidate_id[self.candidate_num,:].unsqueeze(1)],dim=1)}

In [28]:
class MarginRankingLoss():      
    
    def __init__(self, margin, score=None, summary_score=None):
        super(MarginRankingLoss, self).__init__()
        # self._init_param_map(score=score, summary_score=summary_score)
        self.margin = margin
        self.loss_func = torch.nn.MarginRankingLoss(margin)

    def get_loss(self, score, summary_score):
        
        # equivalent to initializing TotalLoss to 0
        # here is to avoid that some special samples will not go into the following for loop
        ones = torch.ones(score.size()).cuda(score.device)
        loss_func = torch.nn.MarginRankingLoss(0.0)
        TotalLoss = loss_func(score, score, ones)

        # candidate loss
        n = score.size(1)
        for i in range(1, n):
            pos_score = score[:, :-i]
            neg_score = score[:, i:]
            pos_score = pos_score.contiguous().view(-1)
            neg_score = neg_score.contiguous().view(-1)
            ones = torch.ones(pos_score.size()).cuda(score.device)
            loss_func = torch.nn.MarginRankingLoss(self.margin * i)
            TotalLoss += loss_func(pos_score, neg_score, ones)

        # gold summary loss
        pos_score = summary_score.unsqueeze(-1).expand_as(score)
        neg_score = score
        pos_score = pos_score.contiguous().view(-1)
        neg_score = neg_score.contiguous().view(-1)
        ones = torch.ones(pos_score.size()).cuda(score.device)
        loss_func = torch.nn.MarginRankingLoss(0.0)
        TotalLoss += loss_func(pos_score, neg_score, ones)
        
        return TotalLoss

In [30]:
class rdass:
  def __init__(self,encoder,device):
    self.encoder = encoder
    for param in self.encoder.parameters():
        param.requires_grad = False
  
  def __call__(self, text_ids = None, label_ids = None, answer_ids = None):
    vector_text = self.encoder(text_ids).detach()['hidden_states'][-1][0,:] # vector_d
    vector_label = self.encoder(label_ids).detach()['hidden_states'][-1][0,:] # vector_r
    vector_answer = self.encoder(answer_ids).detach()['hidden_states'][-1][0,:] # vector_p

    return get_score(vector_text,vector_label,vector_answer)

### Model-Load & train code

- Encoder -> KoBART
- GLM 을 제외한 제일 성능 좋은 모델이고 한국어로 train이 되어 있어 선정함

In [74]:
model = BartModel.from_pretrained(get_pytorch_kobart_model())
summary_model = MatchSum(encoder = model, candidate_num = 5,device = device, hidden_size=768) 

# model_for_eval = AutoModel.from_pretrained("klue/roberta-small")
# metric = rdass(model_for_eval,device)

N_EPOCHS = 3
optimizer = SGD(model.parameters(),lr =0.0001)
scheduler = CosineAnnealingWarmRestarts(optimizer,T_0 = len(train_dataloader)//10,T_mult = 500)
criterion = MarginRankingLoss(margin = 0.01)

using cached model. /content/.cache/kobart_base_cased_ff4bda5738.zip


In [ ]:
model.to(device)
summary_model.to(device)
wandb.init(project='summarization', entity='tkdlqh2')

for epoch in range(N_EPOCHS):
    
    print(f"*****Epoch {epoch} Train Start*****")
    print(f"*****Epoch {epoch} Total Step {len(train_dataloader)}*****")
    total_loss, batch_loss, batch_step = 0,0,0
    model.train()

    for step, batch in enumerate(train_dataloader):
        batch_step+=1
        text_input_ids = batch["text_input_ids"].to(device)        
        label_input_ids = batch["labels_input_ids"].to(device)

        model.zero_grad()
        optimizer.zero_grad()

        # forward
        output = summary_model.forward(text_input_ids, label_input_ids,batch["sentence_input_ids"])
        loss = criterion.get_loss(score = output["score"],summary_score = output["summary_score"])

        # loss 계산
        loss.backward()
        # optimizer 업데이트
        optimizer.step()
        # scheduler 업데이트
        scheduler.step()

        batch_loss += loss.item()
        total_loss += loss.item()

        learning_rate = optimizer.param_groups[0]['lr']
        wandb.log({'train/lr':learning_rate,"train/loss":loss.item()})

        if (step%50 == 0) and (step!=0):
            print(f"Step: {step} Loss: {batch_loss/batch_step:.4f} lr: {optimizer.param_groups[0]['lr']:.4f}")
            # 변수 초기화    
            batch_loss, batch_step = 0,0

    print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
    
    # with torch.no_grad():
    #   print('**Calculating validation results...**')
    #   total_metric, batch_step = 0,0
    #   model.eval()
    #   for step, batch in enumerate(train_dataloader):
    #       batch_step+=1
    #       text_input_ids = batch["text_input_ids"].to(device)        
    #       label_input_ids = batch["labels_input_ids"].to(device)

    #       # forward
    #       output = summary_model.forward(text_input_ids, label_input_ids,batch["sentence_input_ids"])
    #       metric_val = metric(text_input_ids,label_input_ids,output["golden_summary"])

    #       total_metric += metric_val.item()
    # print(f"Epoch {epoch} Total Mean Score : {total_metric/(step+1):.4f}")
    
    print(f"*****Epoch {epoch} Train Finished*****\n")
    torch.save(model.state_dict,f"/content/drive/MyDrive/NLP/kobart_model_{epoch}epoch.pth")

train/loss,█▆▇▅▆▅▄▅▄▄▄▃▃▂▂▂▂▂▃▂▃▂▂▂▂▁▂▁▁▂▂▁▂▁▁▁▁▂▂▁
train/lr,█▇▆▅▃▁▁█████████████████████████████████
train/loss,0.10603
train/lr,0.0001


*****Epoch 0 Train Start*****
*****Epoch 0 Total Step 3500*****
